In [4]:
#8_27 과제 - 영화 정보 얻기
from langchain.chat_models import ChatOpenAI
from langchain.prompts.prompt import PromptTemplate
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(
  temperature=0.1,
  streaming=True,
  callbacks=[StreamingStdOutCallbackHandler()]
)

examples = [
    {
        "question": "영화 '파묘'에 대해 알려주세요.",
        "answer": """
        <영화 정보>
        감독: 장재현
        출연진: 최민식, 김고은, 유해진
        예산: 150억원
        흥행수익: 1237억원
        장르: 미스터리
        시놉시스: 미국 LA, 거액의 의뢰를 받은 무당 ‘화림’(김고은)과 ‘봉길’(이도현)은 기이한 병이 대물림되는 집안의 장손을 만난다. 조상의 묫자리가 화근임을 알아챈 ‘화림’은 이장을 권하고, 돈 냄새를 맡은 최고의 풍수사 ‘상덕’(최민식)과 장의사 ‘영근’(유해진)이 합류한다. “전부 잘 알 거야… 묘 하나 잘못 건들면 어떻게 되는지” 절대 사람이 묻힐 수 없는 악지에 자리한 기이한 묘. ‘상덕’은 불길한 기운을 느끼고 제안을 거절하지만, ‘화림’의 설득으로 결국 파묘가 시작되고… 나와서는 안될 것이 나왔다.
        """,
    },
    {
        "question": "영화 '샹치와 텐 링즈의 전설'에 대해 알려주세요.",
        "answer": """
        <영화 정보>
        감독: 데스틴 크리튼
        출연진: 시무 리우, 양조위, 아콰피나
        예산: 1500만달러
        흥행수익: 3억3,100만 달러
        장르: 액션
        시놉시스: 초인적인 능력을 가진 ‘텐 링즈’의 힘으로 수세기 동안 어둠의 세상을 지배해 온 ‘웬우’ '샹치’는 아버지 ‘웬우’ 밑에서 암살자로 훈련을 받았지만 이를 거부하고 평범함 삶을 선택한다. 그러나 ‘샹치’는 목숨을 노리는 자들의 습격으로 더 이상 운명을 피할 수 없다는 것을 직감하고, 어머니가 남긴 가족의 비밀과 내면의 신비한 힘을 일깨우게 된다 벗어나고 싶은 과거이자, 그 누구보다 두려운 아버지 ‘웬우’를 마주해야 하는 ‘샹치’ 악이 될 것인가? 구원이 될 것인가? 마블의 새로운 시대, 세상에 없던 힘이 탄생한다!
        """,
    },
    {
        "question": "영화 '부산행'에 대해 알려주세요.",
        "answer": """
        <영화 정보>
        감독: 연상호
        출연진: 공유, 정유미, 마동석
        예산: 115억원
        흥행수익: 95,699,124달러
        장르: 호러
        시놉시스: 정체불명의 바이러스가 전국으로 확산되고 대한민국 긴급재난경보령이 선포된 가운데, 열차에 몸을 실은 사람들은 단 하나의 안전한 도시 부산까지 살아가기 위한 치열한 사투를 벌이게 된다. 서울에서 부산까지의 거리 442KM 지키고 싶은, 지켜야만 하는 사람들의 극한의 사투!

        """,
    },
]


example_prompt = PromptTemplate.from_template("Human: {question}\nAI:{answer}")

prompt = FewShotPromptTemplate(
  example_prompt=example_prompt,
  examples=examples,
  suffix="Human: 영화 {movie}에 대해 알려주세요.",
  input_variables=["movie"]
)

chain = prompt | chat

chain.invoke({"movie":"인사이드아웃"})



AI:
        <영화 정보>
        감독: 피트 닥터
        출연진: 애미 파울러, 필리스 스미스, 민디 캘링
        예산: 175백만 달러
        흥행수익: 8억 56백만 달러
        장르: 애니메이션, 코미디
        시놉시스: 소녀 라일리의 머릿속에는 감정들이 살고 있다. 기쁨, 슬픔, 분노, 혐오, 두려움 등 다섯 감정들이 라일리의 삶을 이끌어가는데, 어느 날 가족이 새로운 도시로 이사를 가게 되면서 라일리의 감정들은 혼란스러운 상황에 직면하게 된다. 이야기는 라일리의 머릿속에서 벌어지는 감정들의 모험을 통해 성장과 감정의 중요성을 다룬다.

AIMessageChunk(content='AI:\n        <영화 정보>\n        감독: 피트 닥터\n        출연진: 애미 파울러, 필리스 스미스, 민디 캘링\n        예산: 175백만 달러\n        흥행수익: 8억 56백만 달러\n        장르: 애니메이션, 코미디\n        시놉시스: 소녀 라일리의 머릿속에는 감정들이 살고 있다. 기쁨, 슬픔, 분노, 혐오, 두려움 등 다섯 감정들이 라일리의 삶을 이끌어가는데, 어느 날 가족이 새로운 도시로 이사를 가게 되면서 라일리의 감정들은 혼란스러운 상황에 직면하게 된다. 이야기는 라일리의 머릿속에서 벌어지는 감정들의 모험을 통해 성장과 감정의 중요성을 다룬다.')

In [ ]:
chat.predict("How do you make italian pasta?")